In [1]:
from sdgx.data_models.metadata import Metadata
from sdgx.data_processors.base import DataProcessor
from sdgx.data_processors.manager import DataProcessorManager
import pandas as pd
from typing_extensions import Generator
import numpy as np
from sdgx.data_loader import DataLoader
from sdgx.data_connectors.generator_connector import GeneratorConnector

'''
pd.concat([
        pd.DataFrame([*np.linspace(0, 1, 15000), None], columns=['x']),
        pd.DataFrame([* ([None] * 1000), *([np.nan]*1000)], columns=['y'])
    ],axis=1) 
    
pd.DataFrame([str("".join([chr(i) for i in random.choices(range(int(ord('A')), ord('A') + 26), k=10)])) for j in range(10000)], columns=['z']),
'''
import faker
import random
ff = faker.Faker()
def gener():
    yield pd.concat([
        pd.DataFrame([ff.date() for i in range(15000)], columns=['date']),
        # pd.DataFrame([*np.linspace(0, 1, 15000), None], columns=['x']),
        # pd.DataFrame([*np.linspace(0, 1, 10000), *([np.nan] * 5000)], columns=['y'])
    ], axis=1)
    # D = [*np.linspace(0, 1, 500), *([np.nan]*500), *[None]*100]
    # A = [*[None]*50, *([np.nan]*450), *np.linspace(0, 1000, 450), *[None]*150]
    # yield pd.DataFrame([
    #     [a, None if i>15 else random.random(), d, chr(i%26+67)] for i,(a, d) in enumerate(zip(A, D)) 
    # ], columns=['aax', 'aay', 'z', 'w'])


data_processors_manager = DataProcessorManager()
data_processors = data_processors_manager.registed_default_processor_list
data_processors = [
    (
        d if isinstance(d, DataProcessor) else data_processors_manager.init_data_processor(d)
    )
    for d in data_processors
]
dataconnector = GeneratorConnector(gener)
dataloader = DataLoader(dataconnector)

metadata = Metadata.from_dataloader(dataloader)
metadata.datetime_columns = ["date"]
metadata.discrete_columns = []
metadata.datetime_format = {
    "date": "%Y-%m-%d"
}
for key in dir(metadata):
    if key.endswith('_columns') and key != 'get_all_data_type_columns':
        print(f"{key}={getattr(metadata, key)}")
print(metadata._extend)

for d in data_processors:
    d.fit(metadata=metadata, tabular_data=dataloader)


def chunk_generator() -> Generator[pd.DataFrame, None, None]:
    for chunk in dataloader.iter():
        for d in data_processors:
            chunk = d.convert(chunk)
            
        # assert not chunk.isna().any().any()
        # assert not chunk.isnull().any().any()
        yield chunk


processed_dataloader = DataLoader(
    GeneratorConnector(chunk_generator),
    identity=dataloader.identity
)

processed_dataloader.load_all()

2024-11-22 16:13:03.763 | INFO     | sdgx.data_models.metadata:from_dataloader:294 - Inspecting metadata...
2024-11-22 16:13:04.485 | INFO     | sdgx.data_models.metadata:update_primary_key:508 - Primary Key updated: set().
2024-11-22 16:13:04.486 | INFO     | sdgx.data_processors.transformers.nan:fit:71 - NonValueTransformer Fitted.
2024-11-22 16:13:04.487 | INFO     | sdgx.data_processors.transformers.nan:fit:87 - NonValueTransformer get int columns: set().
2024-11-22 16:13:04.487 | INFO     | sdgx.data_processors.transformers.nan:fit:96 - NonValueTransformer get float columns: set().
2024-11-22 16:13:04.488 | INFO     | sdgx.data_processors.transformers.nan:fit:101 - NonValueTransformer get column list from metadata: ['date'].
2024-11-22 16:13:04.488 | INFO     | sdgx.data_processors.transformers.outlier:fit:70 - OutlierTransformer Fitted.
2024-11-22 16:13:04.489 | INFO     | sdgx.data_processors.formatters.int:fit:40 - IntValueFormatter Fitted.
2024-11-22 16:13:04.489 | INFO     | 

bool_columns=set()
const_columns=set()
datetime_columns=['date']
discrete_columns=[]
float_columns=set()
id_columns=set()
int_columns=set()
pii_columns=set()
defaultdict(<class 'set'>, {'datetime_formats': {'date': '%Y-%m-%d'}, 'numeric_format': {'positive': [], 'negative': []}})


,date
0,3.461760e+08
1,7.311744e+08
2,1.729526e+09
3,5.068800e+07
4,1.003680e+09
...,...
14995,NaN
14996,NaN
14997,NaN
14998,NaN


In [2]:
# pd.concat([
#     pd.DataFrame([*np.linspace(0, 1, 15000), None], columns=['x']),
#     pd.DataFrame([*([None] * 1000), *([np.nan] * 1000)], columns=['y'])
# ], axis=1)

In [3]:
# for i in gener():
#     t = i
# t